In [41]:
import numpy as np
from sklearn.cluster import SpectralClustering,KMeans
from sklearn.metrics import average_precision_score
from sklearn.metrics import roc_auc_score,f1_score
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import roc_curve
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import matthews_corrcoef,accuracy_score,precision_score,recall_score,confusion_matrix
from scipy.stats import pearsonr
import matplotlib.pyplot as plt
import os
import pandas as pd
from pathlib import Path

In [7]:
# HL: transfer from dataden "/umms-kinfai/duolin/ying/reditools2_candidates/"
covfilename=5
datatype='HEK293T_WT_directRNA'

hash_candidat={}
hash_candidat['AFG-H1_directRNA']='H1-AFG.candidate_sites.tab'
hash_candidat['AFG-H9_directRNA']='H9-AFG.candidate_sites.tab'
hash_candidat["PGC-H1_directRNA"]='H1-PGC.candidate_sites.tab'
hash_candidat["DE-H1_directRNA"]='H1-DE.candidate_sites.tab'
hash_candidat["DE-H9_directRNA"]='H9-DE.candidate_sites.tab'
hash_candidat["GM12878_directRNA"]='GM12878.candidate_sites.tab'
hash_candidat["H1-hESC_directRNA"]='H1-hESC.candidate_sites.tab'
hash_candidat["H9-hESC_directRNA"]='H9-hESC.candidate_sites.tab'
hash_candidat['HEK293T_DKO_directRNA']='HEK293T_WT.candidate_sites.tab'
hash_candidat["HEK293T_WT_directRNA"]='HEK293T_WT.candidate_sites.tab'
hash_candidat["HEK_WT_pass"]='HEK293T_WT.candidate_sites.tab'
candidatefile="/nfs/turbo/umms-kinfai/haorli/20240314_ReDD_result_data/figure2a/reditools2_candidates/"+hash_candidat[datatype]


In [3]:
# HL: define ground truth
negative_site = "/nfs/turbo/umms-kinfai/haorli/20240314_ReDD_result_data/figure3a/negative_site/test_HEK293T_WT_negative_sites_rthred0.1.txt"
positive_site = "/nfs/turbo/umms-kinfai/haorli/20240314_ReDD_result_data/figure3a/positive_site/test_HEK293T_WT_positive_sites_rthred0.1.txt"

using_mean_prediction_score=True
if using_mean_prediction_score:
   score_index=-1
else:
   score_index=-2

true_candidate_ratios={}
input = open(candidatefile,'r')
for line in input:
    chr_ = line.split()[0]
    pos_ = line.split()[1].strip()
    true_candidate_ratios[chr_+"-"+pos_]=float(line.split("\t")[3])

nonmodification_sites={}
input = open(negative_site,'r')
for line in input:
    chr_ = line.split("-")[0]
    pos_ = line.split("-")[1].strip()
    nonmodification_sites[chr_+"-"+pos_]=1

positive_sites={}
input = open(positive_site,'r')
for line in input:
    chr_ = line.split("-")[0]
    pos_ = line.split("-")[1].strip()
    positive_sites[chr_+"-"+pos_]=1

In [14]:
foldername="/nfs/turbo/umms-kinfai/haorli/20240314_ReDD_result_data/supfig2/ReDD_results/"
modelist=[foldername+'/Eventfeature_5feature_win5/hg38retrain_HEK293T_KO1_chrsplitHEK293T_WT_directRNA_epochs1_100_epochs2_50',
          foldername+'/Eventfeature_5feature_win7/hg38retrain_HEK293T_KO1_chrsplitHEK293T_WT_directRNA_epochs1_100_epochs2_50',
foldername+'/Eventfeature_3feature_win9/hg38retrain_HEK293T_KO1_chrsplitHEK293T_WT_directRNA_epochs1_100_epochs2_50',
foldername+'/Eventfeature_5feature_win9/ablationseq0011_nopretrain_HEK293T_KO1_chrsplitHEK293T_WT_directRNA_epochs2_50',
foldername+'/Eventfeature_5feature_win9/ablationseq1011_pretrain_HEK293T_KO1_chrsplitHEK293T_WT_directRNAepochs1_100_epochs2_50',
foldername+'/Eventfeature_5feature_win9/ablationseq0111_pretrain_HEK293T_KO1_chrsplitHEK293T_WT_directRNAepochs1_100_epochs2_50',
foldername+'/Eventfeature_5feature_win9/hg38retrain_HEK293T_KO1_chrsplitHEK293T_WT_directRNA_epochs1_100_epochs2_88',
foldername+'/Eventfeature_5feature_win9/ablationnoKO_HEK293T_KO1_chrsplitHEK293T_WT_directRNAepochs1_100_epochs2_50',
]
modelnames=['5D_5mer','5D_7mer','3D_9mer','editing_status_loss_only','ref_loss','basecall_loss','ReDD','REDD_WT_only']

In [49]:
def get_curves(true_value_selected,predict_value_selected):
    # roc
    fpr, tpr,thres = roc_curve(true_value_selected, predict_value_selected)
    sel_fpr = fpr[np.abs(fpr - 0.05).argmin()]
    sel_tpr = tpr[np.abs(fpr - 0.05).argmin()]
    sel_roc_thres = thres[np.abs(fpr - 0.05).argmin()]
    roc_auc = auc(fpr, tpr)
    # pr
    precision, recall, thresholds = precision_recall_curve(true_value_selected, predict_value_selected)
    sel_precision = precision[np.abs(precision - 0.95).argmin()]
    sel_recall =  recall[np.abs(precision - 0.95).argmin()]
    self_pr_thres = thresholds[np.abs(precision - 0.95).argmin()]
    
    precision_roc_auc = average_precision_score(true_value_selected, predict_value_selected)
    
    ratio_threshod = thres[np.argmin(np.absolute(fpr-0.05))]
    predict_value_selected = (predict_value_selected>=ratio_threshod).astype(int)
    tn, fp, fn, tp = confusion_matrix(true_value_selected, predict_value_selected).ravel()
    acc = (tp+tn)/(tp+tn+fp+fn)
    f1 = f1_score(true_value_selected, predict_value_selected)
    return fpr,tpr,thres,roc_auc,sel_fpr,sel_tpr,sel_roc_thres,precision,recall,thresholds,precision_roc_auc,sel_precision,sel_recall,self_pr_thres,acc,f1

In [50]:
# HL: for each model
long_reads_min_coverage = coverage_cutoff = 5
sel_rows = []
for model,met in zip(modelist,modelnames):
    filename1=model+"/test_"+datatype+"_onlycandidate_cov"+str(covfilename)+"_ratio0_modcov0.sitelev.bed" #background must use no filter except coverage
    filename2=model+"/test_"+datatype+"_noncandidate_cov"+str(covfilename)+"_ratio0_modcov0.sitelev.bed"
    true_value_all={}
    predict_value_all={} #909
    num_candidate=0
    #index_neg=[]
    inputs=[open(filename1),open(filename2)]
    for input in inputs:
      for line in input:
        readcoverage=int(line.split()[3])
        if readcoverage<coverage_cutoff:
             continue

        chr_ = line.split()[0]
        pos_ = line.split()[1]
        if len(line.split("\t")) == 6:
            predictscore=float(line.split("\t")[score_index]) #last score is mean prediciton score
        else:
            predictscore=float(line.split("\t")[-1]) #for old result there is no mean prediction scores

        if chr_+"-"+pos_ in positive_sites:
             true_value_all[chr_+"-"+pos_]=true_candidate_ratios[chr_+"-"+pos_]
        elif chr_+"-"+pos_ in nonmodification_sites:
             true_value_all[chr_+"-"+pos_]=0
             #index_neg.append(len(true_value_all)-1)
        else:
             continue

        predict_value_all[chr_+"-"+pos_]=predictscore
    #merge all results using common sites
    true_value_selected=[]
    predict_value_selected=[]
    coord_list = []
    for key in true_value_all:
        true_value_selected.append(true_value_all[key])
        predict_value_selected.append(predict_value_all[key])
        coord_list.append(key)

    true_value_selected=np.asarray(true_value_selected)
    predict_value_selected=np.asarray(predict_value_selected)
    coord_list = np.asarray(coord_list)
    pos_coverage_ratio=0.1

    # # # balance pos and neg
    pos_index=np.where(true_value_selected>=pos_coverage_ratio)[0]
    neg_index=np.where(true_value_selected==0)[0]
    coord_selected = np.concatenate([coord_list[pos_index],coord_list[neg_index]])
    true_value_selected=np.concatenate([true_value_selected[pos_index],true_value_selected[neg_index]])
    predict_value_selected=np.concatenate([predict_value_selected[pos_index],predict_value_selected[neg_index]])
    
    true_value_selected = (np.asarray(true_value_selected) >= pos_coverage_ratio).astype(int) 
    # get ROC and PR curves
    fpr,tpr,roc_thres,roc_auc,sel_fpr,sel_tpr,sel_roc_thres,precision,recall,pr_thres,precision_roc_auc,sel_precision,sel_recall,self_pr_thres,acc,f1 = get_curves(true_value_selected,predict_value_selected)
    roc_df = pd.DataFrame([fpr,tpr,roc_thres]).T
    roc_df.columns = ['FPR','TPR','ROC_thres']
    pr_df = pd.DataFrame([precision,recall,pr_thres]).T
    pr_df.columns = ['Precision','Recall','PR_thres']
    Path(f'plot_data/ROC/').mkdir(exist_ok=True,parents=True)
    Path(f'plot_data/PR/').mkdir(exist_ok=True,parents=True)
    roc_df.to_csv(f'plot_data/ROC/{met}.tsv',sep='\t',index=False)
    pr_df.to_csv(f'plot_data/PR/{met}.tsv',sep='\t',index=False)
    sel_rows.append([met,roc_auc,precision_roc_auc,sel_fpr,sel_tpr,sel_roc_thres,sel_precision,sel_recall,self_pr_thres,acc,f1])


In [51]:
summary_df = pd.DataFrame(sel_rows,columns=['Model','ROC_AUC','PR_AUC','FPR','TPR','ROC_threshold','Precision','Recall','PR_threshold','Accuracy','F1'])
summary_df.to_csv('plot_data/site_level_metrics.tsv',sep='\t',index=False)

In [52]:
summary_df

,Model,ROC_AUC,PR_AUC,FPR,TPR,ROC_threshold,Precision,Recall,PR_threshold,Accuracy,F1
0,5D_5mer,0.871345,0.885675,0.050521,0.582999,0.092,0.949429,0.466720,0.134,0.766239,0.713795
1,5D_7mer,0.890068,0.907216,0.049719,0.700882,0.057,0.948097,0.659182,0.071,0.825581,0.800733
2,3D_9mer,0.861103,0.876545,0.050521,0.696071,0.017,0.950289,0.659182,0.027,0.822775,0.797062
3,editing_status_loss_only,0.861883,0.886645,0.050521,0.671211,0.052,0.949749,0.606255,0.075,0.810345,0.779693
4,ref_loss,0.885121,0.900302,0.049719,0.734563,0.026,0.949733,0.712109,0.033,0.842422,0.823371
5,basecall_loss,0.883158,0.901757,0.050521,0.726544,0.027,0.949134,0.703288,0.036,0.838011,0.817690
6,ReDD,0.896559,0.915097,0.049719,0.730553,0.051,0.950111,0.687249,0.068,0.840417,0.820721
7,REDD_WT_only,0.794234,0.822709,0.049719,0.573376,0.032,0.950938,0.528468,0.064,0.761828,0.706522
